# Práctica SQL - Parte 2: Conceptos Avanzados con Northwind

En esta práctica vamos a trabajar con conceptos más avanzados de SQL usando la misma base de datos Northwind. Veremos:

- HAVING
- Subconsultas
- Funciones de Ventana (RANK, PARTITION BY, ROW_NUMBER)
- LAG y LEAD
- Common Table Expressions (CTE)

## Esquema de la Base de Datos

![Esquema Northwind](https://miro.medium.com/v2/resize:fit:944/1*Qn-ac6Va4Oa0vLsH1suSaw.png)

Primero, instalemos las librerías necesarias:


In [1]:
!pip install ipython-sql==0.3.9 prettytable==3.5.0 sqlalchemy==1.4.46 --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext sql
%sql sqlite:///northwind.db

'Connected: @northwind.db'

# Ejemplos de Conceptos Avanzados

## 1. HAVING
HAVING nos permite filtrar grupos después de agrupar, a diferencia de WHERE que filtra filas individuales antes de agrupar.

Veamos un ejemplo: Encontrar las categorías que tienen más de 10 productos con un precio unitario mayor a $10:


In [7]:
%%sql
SELECT 
    c.CategoryName,
    COUNT(*) as CantidadProductos,
    AVG(p.UnitPrice) as PrecioPromedio
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
WHERE p.UnitPrice > 10
GROUP BY c.CategoryID, c.CategoryName
HAVING COUNT(*) > 10;


 * sqlite:///northwind.db
Done.


CategoryName,CantidadProductos,PrecioPromedio
Condiments,11,24.25


## 2. Subconsultas
Las subconsultas son consultas anidadas dentro de otras consultas. Pueden usarse en diferentes partes de la consulta principal.

### 2.1 Subconsulta en WHERE
Encontremos los productos cuyo precio es mayor al promedio:


In [10]:
%%sql
SELECT ProductName, UnitPrice
FROM Products
WHERE UnitPrice > (
    SELECT AVG(UnitPrice)
    FROM Products
)
ORDER BY UnitPrice DESC
LIMIT 5;


 * sqlite:///northwind.db
Done.


ProductName,UnitPrice
Côte de Blaye,263.5
Thüringer Rostbratwurst,123.79
Mishi Kobe Niku,97
Sir Rodney's Marmalade,81
Carnarvon Tigers,62.5


### 2.2 Subconsulta en FROM
Podemos usar una subconsulta como si fuera una tabla. Veamos el total de ventas por categoría desde el 2023 hasta ahora:


In [11]:
%%sql
SELECT 
    ventas_categoria.CategoryName,
    ventas_categoria.TotalVentas
FROM (
    SELECT 
        c.CategoryName,
        SUM(od.Quantity * od.UnitPrice) as TotalVentas
    FROM Categories c
    JOIN Products p ON c.CategoryID = p.CategoryID
    JOIN "Order Details" od ON p.ProductID = od.ProductID
    JOIN Orders o ON od.OrderID = o.OrderID
    WHERE o.OrderDate >= '2023-01-01'
    GROUP BY c.CategoryID, c.CategoryName
) as ventas_categoria
ORDER BY ventas_categoria.TotalVentas DESC;


 * sqlite:///northwind.db
Done.


CategoryName,TotalVentas
Beverages,6758278.5
Confections,4923889.9799999995
Meat/Poultry,4740726.67
Dairy Products,4221091.9
Condiments,4160110.85
Seafood,3744024.4
Produce,2403984.95
Grains/Cereals,2102382.75


## 3. Funciones de Ventana
Las funciones de ventana nos permiten realizar cálculos a través de un conjunto de filas relacionadas con la fila actual.

### 3.1 RANK()
Veamos el ranking de empleados por total de ventas:


In [12]:
%%sql
SELECT 
    e.FirstName || ' ' || e.LastName as Empleado,
    ROUND(SUM(od.Quantity * od.UnitPrice), 2) as TotalVentas,
    RANK() OVER (ORDER BY SUM(od.Quantity * od.UnitPrice) DESC) as Ranking
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, e.FirstName, e.LastName;


 * sqlite:///northwind.db
Done.


Empleado,TotalVentas,Ranking
Margaret Peacock,51505691.8,1
Steven Buchanan,51393234.57,2
Janet Leverling,50455812.22,3
Nancy Davolio,49669459.34,4
Robert King,49668627.06,5
Laura Callahan,49287575.56,6
Michael Suyama,49144251.53,7
Anne Dodsworth,49025334.37,8
Andrew Fuller,48325312.27,9


### 3.2 PARTITION BY
PARTITION BY nos permite dividir los datos en grupos y aplicar funciones de ventana a cada grupo por separado.

Veamos el ranking de productos por ventas dentro de cada categoría:


In [ ]:
%%sql
WITH RankingPorCategoria AS (
    SELECT 
        c.CategoryName,
        p.ProductName,
        ROUND(SUM(od.Quantity * od.UnitPrice), 2) AS TotalVentas,
        RANK() OVER (
            PARTITION BY c.CategoryID 
            ORDER BY SUM(od.Quantity * od.UnitPrice) DESC
        ) AS RankingEnCategoria
    FROM Categories c
    JOIN Products p ON c.CategoryID = p.CategoryID
    JOIN "Order Details" od ON p.ProductID = od.ProductID
    GROUP BY c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
)
SELECT *
FROM RankingPorCategoria
WHERE RankingEnCategoria <= 1
ORDER BY CategoryName, RankingEnCategoria;

 * sqlite:///northwind.db
Done.


CategoryName,ProductName,TotalVentas,RankingEnCategoria
Beverages,Côte de Blaye,53274482.7,1
Beverages,Ipoh Coffee,9334927.2,2
Beverages,Chang,3832714.2,3
Condiments,Vegie-spread,8811700.4,1
Condiments,Northwoods Cranberry Sauce,8074640.0,2
Condiments,Sirop d'érable,5841895.8,3
Confections,Sir Rodney's Marmalade,16654879.8,1
Confections,Tarte au sucre,9955529.0,2
Confections,Schoggi Schokolade,8829210.1,3
Dairy Products,Raclette Courdavault,11221551.0,1


### 3.3 LAG y LEAD
LAG nos permite acceder a datos de la fila anterior, mientras que LEAD nos permite acceder a datos de la fila siguiente.

Veamos cómo varía el total de ventas mes a mes, mostrando el mes anterior y el siguiente:


In [15]:
%%sql
WITH ventas_mensuales AS (
    SELECT 
        strftime('%Y-%m', OrderDate) as Mes,
        ROUND(SUM(od.Quantity * od.UnitPrice), 2) as TotalVentas
    FROM Orders o
    JOIN "Order Details" od ON o.OrderID = od.OrderID
    GROUP BY strftime('%Y-%m', OrderDate)
)
SELECT 
    Mes,
    TotalVentas,
    LAG(TotalVentas) OVER (ORDER BY Mes) as VentasMesAnterior,
    LEAD(TotalVentas) OVER (ORDER BY Mes) as VentasMesSiguiente
FROM ventas_mensuales
ORDER BY Mes DESC
LIMIT 5;


 * sqlite:///northwind.db
Done.


Mes,TotalVentas,VentasMesAnterior,VentasMesSiguiente
2023-10,2923364.35,3544698.51,None
2023-09,3544698.51,3293158.67,2923364.35
2023-08,3293158.67,3350337.36,3544698.51
2023-07,3350337.36,3071787.73,3293158.67
2023-06,3071787.73,3896544.35,3350337.36


Nota: El dataset llega hasta el mes octubre de 2023, por eso no hay un dato en el mes siguiente de ese y da "None"

# Ejercicios

A continuación se encuentran algunos ejercicios con estos conceptos más avanzados. Intentá resolver los siguientes ejercicios por tu cuenta. Las soluciones están al final del notebook.

1. Usando HAVING, encontrá los clientes que hicieron más de 100 órdenes y cuyo promedio de monto por orden sea mayor a $750.


In [ ]:
%%sql


2. Usando una subconsulta en WHERE, encontrá los productos que tienen un precio unitario 5 veces mayor al promedio de su categoría.


In [ ]:
%%sql


3. Usando RANK(), mostrá el top 3 de productos más vendidos por categoría en términos de unidades vendidas.


In [ ]:
%%sql


4. Mostrar los clientes que hayan realizado más órdenes que el promedio de órdenes por cliente en la base, junto con la cantidad total de órdenes que hicieron.

In [ ]:
%%sql


5. Usando una CTE y funciones de ventana, encontrá para cada empleado:
   - Su total de ventas
   - El promedio de ventas de su territorio
   - Qué porcentaje representa sus ventas sobre el total de su territorio

In [ ]:
%%sql


# Soluciones

A continuación se encuentran las soluciones para los ejercicios planteados. Intentá resolverlos por tu cuenta antes de ver las respuestas.

1. Usando HAVING, encontrá los clientes que hicieron más de 100 órdenes y cuyo promedio de monto por orden sea mayor a $750.


In [21]:
%%sql
SELECT 
    c.CompanyName,
    COUNT(DISTINCT o.OrderID) as TotalOrdenes,
    ROUND(AVG(od.Quantity * od.UnitPrice), 2) as MontoPromedioOrden
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY c.CustomerID, c.CompanyName
HAVING COUNT(DISTINCT o.OrderID) > 100
   AND AVG(od.Quantity * od.UnitPrice) > 750
ORDER BY MontoPromedioOrden DESC;

 * sqlite:///northwind.db
Done.


CompanyName,TotalOrdenes,MontoPromedioOrden
Hanari Carnes,180,755.44
Bólido Comidas preparadas,182,754.7
Maison Dewey,162,753.64
Vins et alcools Chevalier,158,752.9
Familia Arquibaldo,168,750.94
Save-a-lot Markets,190,750.73
Rancho grande,194,750.62


2. Usando una subconsulta en WHERE, encontrá los productos que tienen un precio unitario 5 veces mayor al promedio de su categoría.

In [27]:
%%sql
SELECT 
    c.CategoryName,
    p.ProductName,
    p.UnitPrice,
    (
        SELECT ROUND(AVG(p2.UnitPrice), 2)
        FROM Products p2
        WHERE p2.CategoryID = p.CategoryID
    ) AS PromedioCategoria
FROM Products p
JOIN Categories c ON p.CategoryID = c.CategoryID
WHERE p.UnitPrice > (
    SELECT 5 * AVG(p2.UnitPrice)
    FROM Products p2
    WHERE p2.CategoryID = p.CategoryID
)
ORDER BY c.CategoryName, p.UnitPrice DESC;


 * sqlite:///northwind.db
Done.


CategoryName,ProductName,UnitPrice,PromedioCategoria
Beverages,Côte de Blaye,263.5,37.98


3. Usando RANK(), mostrá el top 3 de productos más vendidos por categoría en términos de unidades vendidas.

In [28]:
%%sql
WITH ventas_producto AS (
    SELECT 
        c.CategoryName,
        p.ProductName,
        SUM(od.Quantity) as UnidadesVendidas,
        RANK() OVER (
            PARTITION BY c.CategoryID 
            ORDER BY SUM(od.Quantity) DESC
        ) as Ranking
    FROM Categories c
    JOIN Products p ON c.CategoryID = p.CategoryID
    JOIN "Order Details" od ON p.ProductID = od.ProductID
    GROUP BY c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
)
SELECT *
FROM ventas_producto
WHERE Ranking <= 3
ORDER BY CategoryName, Ranking;

 * sqlite:///northwind.db
Done.


CategoryName,ProductName,UnidadesVendidas,Ranking
Beverages,Outback Lager,204403,1
Beverages,Sasquatch Ale,203667,2
Beverages,Ipoh Coffee,202968,3
Condiments,Louisiana Hot Spiced Okra,206213,1
Condiments,Sirop d'érable,205005,2
Condiments,Aniseed Syrup,202186,3
Confections,Sir Rodney's Marmalade,205637,1
Confections,Teatime Chocolate Biscuits,205487,2
Confections,Gumbär Gummibärchen,204761,3
Dairy Products,Raclette Courdavault,204137,1


4. Mostrar los clientes que hayan realizado más órdenes que el promedio de órdenes por cliente en la base, junto con la cantidad total de órdenes que hicieron.

In [41]:
%%sql
SELECT
    c.CompanyName,
    COUNT(o.OrderID) AS CantidadOrdenes,
    (
        SELECT ROUND(AVG(CantidadOrdenes), 2)
        FROM (
            SELECT COUNT(OrderID) AS CantidadOrdenes
            FROM Orders
            GROUP BY CustomerID
        )
    ) AS PromedioOrdenes
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
GROUP BY c.CustomerID, c.CompanyName
HAVING COUNT(o.OrderID) > (
    SELECT AVG(CantidadOrdenes)
    FROM (
        SELECT COUNT(OrderID) AS CantidadOrdenes
        FROM Orders
        GROUP BY CustomerID
    )
)
ORDER BY CantidadOrdenes DESC;

 * sqlite:///northwind.db
Done.


CompanyName,CantidadOrdenes,PromedioOrdenes
B's Beverages,210,175.08
LILA-Supermercado,203,175.08
Ricardo Adocicados,203,175.08
Gourmet Lanchonetes,202,175.08
Princesa Isabel Vinhos,200,175.08
Hungry Coyote Import Store,198,175.08
Tortuga Restaurante,197,175.08
Ana Trujillo Emparedados y helados,195,175.08
Folies gourmandes,195,175.08
Rancho grande,194,175.08


5. Usando una CTE y funciones de ventana, encontrá para cada empleado:
   - Su total de ventas
   - El promedio de ventas de su territorio
   - Qué porcentaje representa sus ventas sobre el total de su territorio

In [29]:
%%sql
WITH ventas_empleado AS (
    SELECT 
        e.FirstName || ' ' || e.LastName as Empleado,
        e.Region as Territorio,
        ROUND(SUM(od.Quantity * od.UnitPrice), 2) as TotalVentas
    FROM Employees e
    JOIN Orders o ON e.EmployeeID = o.EmployeeID
    JOIN "Order Details" od ON o.OrderID = od.OrderID
    GROUP BY e.EmployeeID, Empleado, e.Region
)
SELECT 
    Empleado,
    Territorio,
    TotalVentas,
    ROUND(AVG(TotalVentas) OVER (PARTITION BY Territorio), 2) as PromedioTerritorio,
    ROUND(TotalVentas * 100.0 / SUM(TotalVentas) OVER (PARTITION BY Territorio), 2) as PorcentajeTerritorio
FROM ventas_empleado
ORDER BY Territorio, TotalVentas DESC;

 * sqlite:///northwind.db
Done.


Empleado,Territorio,TotalVentas,PromedioTerritorio,PorcentajeTerritorio
Steven Buchanan,British Isles,51393234.57,49807861.88,25.8
Robert King,British Isles,49668627.06,49807861.88,24.93
Michael Suyama,British Isles,49144251.53,49807861.88,24.67
Anne Dodsworth,British Isles,49025334.37,49807861.88,24.61
Margaret Peacock,North America,51505691.8,49848770.24,20.66
Janet Leverling,North America,50455812.22,49848770.24,20.24
Nancy Davolio,North America,49669459.34,49848770.24,19.93
Laura Callahan,North America,49287575.56,49848770.24,19.77
Andrew Fuller,North America,48325312.27,49848770.24,19.39
